In [33]:
import qml 

In [34]:
from glob import glob
import numpy as np

In [35]:
database_xyzs = sorted(glob("../qm7/*.xyz"))

In [36]:
database_mols = [qml.Compound(x) for x in database_xyzs]

In [37]:
def get_ncharges_coords(mol):
    ncharges = mol.nuclear_charges
    heavy_ints = [i for i,x in enumerate(ncharges) if x!=1]
    heavy_ncharges = [ncharges[i] for i in heavy_ints]
    coords = mol.coordinates
    heavy_coords = [coords[i] for i in heavy_ints]
    return heavy_ncharges, heavy_coords

In [38]:
ncharges = [x.nuclear_charges for x in database_mols]

In [39]:
mbtypes=qml.representations.get_slatm_mbtypes([[6,7,8,16]])

In [40]:
mbtypes

[[8],
 [16],
 [6],
 [7],
 [8, 8],
 [16, 16],
 [6, 6],
 [7, 7],
 (8, 16),
 (8, 6),
 (8, 7),
 (16, 6),
 (16, 7),
 (6, 7),
 [8, 16, 6],
 [8, 6, 16],
 [16, 8, 6],
 [8, 16, 7],
 [8, 7, 16],
 [16, 8, 7],
 [8, 6, 7],
 [8, 7, 6],
 [6, 8, 7],
 [16, 6, 7],
 [16, 7, 6],
 [6, 16, 7]]

In [41]:
def get_rep(mol, mbtypes):
    ncharges, coords = get_ncharges_coords(mol)
    rep = np.array(qml.representations.generate_slatm(coords, ncharges, 
                                            mbtypes, local=True))
    return ncharges, rep

In [42]:
# pad size is based on largest target 

In [43]:
database_ncharges = []
database_reps = []
for mol in database_mols:
    ncharge, rep = get_rep(mol, mbtypes)
    database_ncharges.append(ncharge)
    database_reps.append(rep)

In [44]:
database_reps = np.array(database_reps)

<ipython-input-44-ff11eb7bdbf5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  database_reps = np.array(database_reps)


In [45]:
database_labels = [t.split("/")[-1].split(".xyz")[0] for t in database_xyzs]

In [46]:
database_labels = np.array(database_labels)

In [47]:
database_ncharges = np.array(database_ncharges)

<ipython-input-47-defeabfdee20>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  database_ncharges = np.array(database_ncharges)


In [48]:
database_reps[0].shape

(1, 3110)

In [49]:
np.savez("../representations/database_SLATM.npz", 
         database_labels=database_labels, 
         database_reps=database_reps,
        database_ncharges=database_ncharges)